<a href="https://colab.research.google.com/github/neeraj-pola/Book-recommendation-using-knn/blob/main/Book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-11-08 16:08:23--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  37.3MB/s    in 0.7s    

2022-11-08 16:08:24 (37.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
print(df_books)
print(df_ratings)

              isbn                                              title  \
0       0195153448                                Classical Mythology   
1       0002005018                                       Clara Callan   
2       0060973129                               Decision in Normandy   
3       0374157065  Flu: The Story of the Great Influenza Pandemic...   
4       0393045218                             The Mummies of Urumchi   
...            ...                                                ...   
271374  0440400988                         There's a Bat in Bunk Five   
271375  0525447644                            From One to One Hundred   
271376  006008667X  Lily Dale : The True Story of the Town that Ta...   
271377  0192126040                        Republic (World's Classics)   
271378  0767409752  A Guided Tour of Rene Descartes' Meditations o...   

                      author  
0         Mark P. O. Morford  
1       Richard Bruce Wright  
2               Carlo D'Este  

In [ ]:
user_ratingCount = (df_ratings.
     groupby(by = ['user'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['user', 'totalRatingCount']]
)

In [ ]:
users_to_remove = user_ratingCount.query('totalRatingCount > 200').user.tolist()

In [ ]:
df = pd.merge(df_ratings,df_books,on='isbn')
df

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,2313,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose
2,6543,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
3,8680,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose
4,10314,034545104X,9.0,Flesh Tones: A Novel,M. J. Rose
...,...,...,...,...,...
1031170,276688,0517145553,0.0,Mostly Harmless,Douglas Adams
1031171,276688,1575660792,7.0,Gray Matter,Shirley Kennett
1031172,276690,0590907301,0.0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey
1031173,276704,0679752714,0.0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman


In [ ]:
book_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
print(book_ratingCount)

                                                    title  totalRatingCount
0        A Light in the Storm: The Civil War Diary of ...                 4
1                                   Always Have Popsicles                 1
2                    Apple Magic (The Collector's series)                 1
3        Ask Lily (Young Women of Faith: Lily Series, ...                 1
4        Beyond IBM: Leadership Marketing and Finance ...                 1
...                                                   ...               ...
241085                                      Ã?Â?lpiraten.                 2
241086                     Ã?Â?rger mit Produkt X. Roman.                 4
241087                                Ã?Â?sterlich leben.                 1
241088                              Ã?Â?stlich der Berge.                 3
241089                                  Ã?Â?thique en toc                 2

[241090 rows x 2 columns]


In [ ]:
rating_with_totalRatingCount = df.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount > 100')

In [ ]:
rating_popular_movie = rating_popular_movie[rating_popular_movie['user'].isin(users_to_remove)]

In [ ]:
book_features_df = rating_popular_movie.pivot_table(index='title',columns='user',values='rating').fillna(0)
book_features_df_matrix = csr_matrix(book_features_df.values)
print(book_features_df_matrix)

  (0, 0)	9.0
  (0, 16)	8.0
  (0, 26)	3.3333333
  (0, 37)	9.0
  (0, 42)	8.0
  (0, 59)	7.0
  (0, 136)	9.0
  (0, 157)	10.0
  (0, 171)	9.0
  (0, 188)	8.0
  (0, 221)	10.0
  (0, 222)	10.0
  (0, 246)	5.0
  (0, 278)	10.0
  (0, 360)	10.0
  (0, 371)	9.0
  (0, 390)	10.0
  (0, 477)	9.0
  (0, 491)	7.0
  (0, 507)	9.0
  (0, 554)	10.0
  (0, 603)	9.0
  (0, 610)	8.0
  (0, 694)	9.0
  (0, 733)	9.0
  :	:
  (903, 16)	8.0
  (903, 26)	3.5
  (903, 86)	10.0
  (903, 104)	2.5
  (903, 112)	5.0
  (903, 151)	6.0
  (903, 234)	7.0
  (903, 291)	8.0
  (903, 304)	9.0
  (903, 323)	10.0
  (903, 368)	10.0
  (903, 385)	9.0
  (903, 446)	7.0
  (903, 450)	9.0
  (903, 485)	7.0
  (903, 490)	3.0
  (903, 498)	10.0
  (903, 505)	8.0
  (903, 544)	10.0
  (903, 592)	8.0
  (903, 699)	10.0
  (903, 712)	8.0
  (903, 744)	10.0
  (903, 829)	9.0
  (903, 879)	8.0


In [ ]:
from sklearn.neighbors import NearestNeighbors
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  model_knn = NearestNeighbors(metric = 'cosine', n_neighbors=5, algorithm='auto')
  model_knn.fit(book_features_df_matrix)
  for query_index in range(len(book_features_df)):
    if book_features_df.index[query_index] == book:
        break

  recommended_books = [book_features_df.index[query_index], []]
  distances, indices = model_knn.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1))
  # now we located the book. lets show the recomendations
  for i in range(1, len(distances.flatten())):
    recommended_books[1].insert(0, [book_features_df.index[indices.flatten()[i]], distances.flatten()[i]])
  

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['I Know This Much Is True', 0.7529293], ['The Lovely Bones: A Novel', 0.7230184], ['Blue Diary', 0.71828747]]]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['I Know This Much Is True', 0.7529293], ['The Lovely Bones: A Novel', 0.7230184], ['Blue Diary', 0.71828747]]]
You passed the challenge! 🎉🎉🎉🎉🎉
